# Federated 2d XRay registration with MONAI

## Introduction

This tutorial shows how to deploy in Fed-BioMed the 2d image classification example provided in the project MONAI (https://monai.io/):

https://github.com/Project-MONAI/tutorials/blob/master/2d_registration/registration_mednist.ipynb

Being MONAI based on PyTorch, the deployment within Fed-BioMed follows seamlessy the same general structure of general PyTorch models. 

Following the MONAI example, this tutorial is based on the MedNIST dataset:

https://github.com/Project-MONAI/MONAI/blob/master/examples/notebooks/mednist_tutorial.ipynb.

## Creating MedNIST nodes

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

To proceed with the tutorial, we created an iid partitioning of the MedNIST dataset between 3 clients. Each client has 3000 image samples for each class. The training partitions are availables at the following link:

https://drive.google.com/file/d/1vLIcBdtdAhh6K-vrgCFy_0Y55dxOWZwf/view

The dataset owned by each client has structure:


└── client_*/

    ├── AbdomenCT/
    
    └── BreastMRI/
    
    └── CXR/
    
    └── ChestCT/
    
    └── Hand/
    
    └── HeadCT/      

To create the federated dataset, we follow the standard procedure for node creation/population of Fed-BioMed. 
After activating the fedbiomed network with the commands

`source ./scripts/fedbiomed_environment network`

and 

`./scripts/fedbiomed_run network`

we create a first node by using the commands

`source ./scripts/fedbiomed_environment node`

`./scripts/fedbiomed_run node start`

We then poulate the node with the data of first client:

`./scripts/fedbiomed_run node add`

We select option 3 (images) to add MedNIST partition of client 1, by just picking the folder of client 1. We use `mednist` as tag to save the selected dataset.
We can further check that the data has been added by executing `./scripts/fedbiomed_run node list`

Following the same procedure, we create the other two nodes with the datasets of client 2 and client 3 respectively.


## Running Fed-BioMed Researcher

We are now ready to start the reseracher enviroment with the command `source ./scripts/fedbiomed_environment researcher`, and open the Jupyter notebook with `./scripts/fedbiomed_run researcher`. 

We can first quesry the network for the mednist dataset. In this case, the nodes are sharing the respective partitions unsing the same tag `mednist`:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-04 10:22:34,515 fedbiomed INFO - Component environment:
2022-01-04 10:22:34,517 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-04 10:22:35,254 fedbiomed INFO - Messaging researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x1089415e0>
2022-01-04 10:22:35,312 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-04 10:22:35,327 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / DEBUG - Message received: {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'command': 'list'}
2022-01-04 10:22:35,332 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / DEBUG - Message received: {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'command': 'list'}
2022-01-04 10:22:35,337 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / DEBUG - Message received: {'researcher_id': 'res

{'node_15981cb3-39ef-4c54-9e72-2432d023491f': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': 'mednist',
   'shape': [16954, 3, 64, 64]}],
 'node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': 'mednist',
   'shape': [18000, 3, 64, 64]}],
 'node_6db91b12-7507-4dd4-b3ea-788da19cc6ed': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': 'mednist',
   'shape': [18000, 3, 64, 64]}]}

## Create an experiment to train a model on the data found

The code for network and data loader of the MONAI tutorial can now be deployed in Fed-BioMed.
We first import the necessary modules from `fedbiomed` and `monai` libraries:

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mednist.py'

In [4]:
from monai.utils import set_determinism, first
from monai.transforms import (
    EnsureChannelFirstD,
    Compose,
    LoadImageD,
    RandRotateD,
    RandZoomD,
    ScaleIntensityRanged,
    EnsureTypeD,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.config import print_config, USE_COMPILED
from monai.networks.nets import GlobalNet
from monai.networks.blocks import Warp
from monai.apps import MedNISTDataset

We can now define the training plan. Note that we use the standard `TorchTrainingPlan` natively provided in Fed-BioMed. We reuse the `MedNISTDataset` data loader defined in the original MONAI tutorial, which is returned by the method `training_data`, which also implements the data parsing from the nodes `dataset_path`. We should also properly define the `training_routine`, following the MONAI tutorial. According to the MONAI tutorial, the model is the `GlobalNet` and the loss is `MSELoss`.

In [5]:
%%writefile "$model_file"

import os
import numpy as np
import torch
from torch.nn import MSELoss
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from fedbiomed.common.logger import logger
from torchvision import datasets, transforms
from typing import Union, List

from monai.utils import set_determinism, first
from monai.transforms import (
    EnsureChannelFirstD,
    Compose,
    LoadImageD,
    RandRotateD,
    RandZoomD,
    ScaleIntensityRanged,
    EnsureTypeD,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.config import print_config, USE_COMPILED
from monai.networks.nets import GlobalNet
from monai.networks.blocks import Warp
from monai.apps import MedNISTDataset

# Here we define the model to be used. 
class MyMonaiTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyMonaiTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["import numpy as np",
                "import os",
                "from fedbiomed.common.logger import logger",
                "from torch.nn import MSELoss",
                "from typing import Union, List",
                "from monai.utils import set_determinism, first",
                "from monai.transforms import (EnsureChannelFirstD,Compose,LoadImageD,RandRotateD,RandZoomD,ScaleIntensityRanged,EnsureTypeD,)",
                "from monai.data import DataLoader, Dataset, CacheDataset",
                "from monai.config import print_config, USE_COMPILED",
                "from monai.networks.nets import GlobalNet",
                "from monai.networks.blocks import Warp",
                "from monai.apps import MedNISTDataset",]
        self.add_dependency(deps)
        
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda:0" if use_cuda else "cpu")
        
        self.model = GlobalNet(
            image_size=(64, 64),
            spatial_dims=2,
            in_channels=2,  # moving and fixed
            num_channel_initial=16,
            depth=3)#.to(self.device)
        self.image_loss = MSELoss()
        
        if USE_COMPILED:
            self.warp_layer = Warp(3, "border").to(self.device)
        else:
            self.warp_layer = Warp("bilinear", "border").to(self.device)
            
        self.optimizer = torch.optim.Adam(self.model.parameters(), 1e-5)
        
    def training_data(self, batch_size = 20):
        # Custom torch Dataloader for MedNIST data
        data_path = self.dataset_path
        # The following line is needed if client structure does not contain the "/MedNIST" folder
        MedNISTDataset.dataset_folder_name = ""
        train_data = MedNISTDataset(root_dir=data_path, section="training", download=False, transform=None)
        training_datadict = [
            {"fixed_hand": item["image"], "moving_hand": item["image"]}
            for item in train_data.data if item["label"] == 4  # label 4 is for xray hands
        ]
        train_transforms = Compose(
            [
                LoadImageD(keys=["fixed_hand", "moving_hand"]),
                EnsureChannelFirstD(keys=["fixed_hand", "moving_hand"]),
                ScaleIntensityRanged(keys=["fixed_hand", "moving_hand"],
                                     a_min=0., a_max=255., b_min=0.0, b_max=1.0, clip=True,),
                RandRotateD(keys=["moving_hand"], range_x=np.pi/4, prob=1.0, keep_size=True, mode="bicubic"),
                RandZoomD(keys=["moving_hand"], min_zoom=0.9, max_zoom=1.1, prob=1.0, mode="bicubic", align_corners=False),
                EnsureTypeD(keys=["fixed_hand", "moving_hand"]),
            ]
        )
        train_ds = CacheDataset(data=training_datadict[:1000], transform=train_transforms,
                                cache_rate=1.0, num_workers=0)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
        
        return train_loader

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, moving, fixed):
        ddf = self.forward(torch.cat((moving, fixed), dim=1))
        pred_image = self.warp_layer(moving, ddf)
        loss = self.image_loss(pred_image, fixed)
        return loss
    
    def training_routine(self,
                         epochs: int = 2,
                         log_interval: int = 10,
                         lr: Union[int, float] = 1e-3,
                         batch_size: int = 48,
                         batch_maxnum: int = 0,
                         dry_run: bool = False,
                         monitor=None):
        
        if self.optimizer is None:
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        self.model.to(self.device)
        
        training_data = self.training_data(batch_size=batch_size)

        for epoch in range(1, epochs + 1):
            self.model.train()
            for batch_idx, batch_data in enumerate(training_data):
                self.optimizer.zero_grad()
                moving = batch_data["moving_hand"].to(self.device)
                fixed = batch_data["fixed_hand"].to(self.device)
                res = self.training_step(moving, fixed)
                res.backward()
                self.optimizer.step()

                # do not take into account more than batch_maxnum
                # batches from the dataset
                if (batch_maxnum > 0) and (batch_idx >= batch_maxnum):
                    logger.debug('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                    break

                if batch_idx % log_interval == 0:
                    logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch,
                        batch_idx * len(moving),
                        len(training_data.dataset),
                        100 * batch_idx / len(training_data),
                        res.item()))

                    # Send scalar values via general/feedback topic
                    if monitor is not None:
                        monitor.add_scalar('Loss', res.item(), batch_idx, epoch)

                    if dry_run:
                        return

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmpfmww9px7/class_export_mednist.py


We now set the model and training parameters. Note that in this case, no model argument is required.

In [6]:
model_args = {}

training_args = {
    'batch_size': 16, 
    'lr': 1e-5, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum':250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

The experiment can be now defined, by providing the `mednist` tag, and running the local training on nodes with model defined in `model_path`, standard `aggregator` (FedAvg) and `client_selection_strategy` (all nodes used). Federated learning is going to be perfomed through 5 optimization rounds.

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['mednist']
rounds = 5

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyMonaiTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None
                )

2022-01-04 10:23:02,866 fedbiomed INFO - Searching dataset with data tags: ['mednist'] for all nodes
2022-01-04 10:23:02,892 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / DEBUG - Message received: {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'tags': ['mednist'], 'command': 'search'}
2022-01-04 10:23:02,893 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / DEBUG - Message received: {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'tags': ['mednist'], 'command': 'search'}
2022-01-04 10:23:02,902 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / DEBUG - Message received: {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'tags': ['mednist'], 'command': 'search'}
2022-01-04 10:23:12,890 fedbiomed INFO - Node selected for training -> node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6
2022-01-04 10:23:12,892 fedbiomed INFO - Node selected for training -> node_15981cb3-39ef-

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [8]:
exp.run()

2022-01-04 10:23:19,026 fedbiomed INFO - Sampled nodes in round 0 ['node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6', 'node_15981cb3-39ef-4c54-9e72-2432d023491f', 'node_6db91b12-7507-4dd4-b3ea-788da19cc6ed']
2022-01-04 10:23:19,028 fedbiomed INFO - Send message to node node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 - {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'job_id': 'f023bb56-34ec-4ba0-bee8-e6d043f45388', 'training_args': {'batch_size': 16, 'lr': 1e-05, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/04/my_model_78e85ccc-2b2c-40ee-9a8c-b6befbeef531.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/04/my_model_48cef847-9bf4-4d28-9f07-5bec030d72f9.pt', 'model_class': 'MyMonaiTrainingPlan', 'training_data': {'node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6': ['dataset_b0a51271-2d4c-47ac-9d3a-e2e38848e63e']}}
2022-01-04 10:23:19,030 fedbiomed DEBUG - researcher_2f6

2022-01-04 10:23:20,953 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / DEBUG - Dataset_path/Users/balelli/Downloads/MedNIST_clients/client_2
2022-01-04 10:23:33,493 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 1 [0/1000 (0%)]	Loss: 0.070645
2022-01-04 10:23:34,650 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 1 [0/1000 (0%)]	Loss: 0.072996
2022-01-04 10:23:35,033 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 1 [0/1000 (0%)]	Loss: 0.086545
2022-01-04 10:23:46,551 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 1 [160/1000 (16%)]	Loss: 0.086790
2022-01-04 10:23:47,356 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 1 [160/1000 (16%)]	Loss: 0.077176
2022-01-04 10:23:47,552 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO

2022-01-04 10:26:26,363 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 3 [640/1000 (63%)]	Loss: 0.045404
2022-01-04 10:26:26,549 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 3 [640/1000 (63%)]	Loss: 0.044202
2022-01-04 10:26:33,874 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 3 [800/1000 (79%)]	Loss: 0.050323
2022-01-04 10:26:34,286 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 3 [800/1000 (79%)]	Loss: 0.043805
2022-01-04 10:26:34,482 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 3 [800/1000 (79%)]	Loss: 0.042520
2022-01-04 10:26:41,858 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 3 [960/1000 (95%)]	Loss: 0.047066
2022-01-04 10:26:42,292 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Trai

2022-01-04 10:26:55,609 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / DEBUG - Message received: {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'job_id': 'f023bb56-34ec-4ba0-bee8-e6d043f45388', 'training_args': {'batch_size': 16, 'lr': 1e-05, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/04/my_model_78e85ccc-2b2c-40ee-9a8c-b6befbeef531.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/04/researcher_params_b2e04134-ee94-4e89-815e-297821474ecf.pt', 'model_class': 'MyMonaiTrainingPlan', 'training_data': {'node_6db91b12-7507-4dd4-b3ea-788da19cc6ed': ['dataset_4d619920-dd9c-4f6a-a308-055ba0c6eed1']}}
2022-01-04 10:26:55,611 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'job_id': 'f023bb56-34ec-4ba0-bee8-e6d043f4

2022-01-04 10:28:36,785 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 2 [800/1000 (79%)]	Loss: 0.037399
2022-01-04 10:28:37,117 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 2 [800/1000 (79%)]	Loss: 0.051408
2022-01-04 10:28:45,238 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 2 [960/1000 (95%)]	Loss: 0.047180
2022-01-04 10:28:45,807 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 2 [960/1000 (95%)]	Loss: 0.042706
2022-01-04 10:28:46,116 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 2 [960/1000 (95%)]	Loss: 0.057680
2022-01-04 10:28:47,498 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 3 [0/1000 (0%)]	Loss: 0.040670
2022-01-04 10:28:48,107 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train E

...

2022-01-04 10:33:08,507 fedbiomed DEBUG - researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc
2022-01-04 10:33:08,508 fedbiomed INFO - Send message to node node_15981cb3-39ef-4c54-9e72-2432d023491f - {'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'job_id': 'f023bb56-34ec-4ba0-bee8-e6d043f45388', 'training_args': {'batch_size': 16, 'lr': 1e-05, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/04/my_model_78e85ccc-2b2c-40ee-9a8c-b6befbeef531.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/04/researcher_params_9f260aac-c0b9-4b99-a0ab-c3a571d154ad.pt', 'model_class': 'MyMonaiTrainingPlan', 'training_data': {'node_15981cb3-39ef-4c54-9e72-2432d023491f': ['dataset_e71735f5-f5c7-4e1e-9d38-712989883da9']}}
2022-01-04 10:33:08,510 fedbiomed DEBUG - researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc
2022-01-04 10:33:08,513 fedbiomed INFO - Send message to node node_6d

2022-01-04 10:33:26,210 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 1 [160/1000 (16%)]	Loss: 0.037334
2022-01-04 10:33:26,353 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 1 [160/1000 (16%)]	Loss: 0.040002
2022-01-04 10:33:33,648 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 1 [320/1000 (32%)]	Loss: 0.032632
2022-01-04 10:33:33,687 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 1 [320/1000 (32%)]	Loss: 0.042346
2022-01-04 10:33:33,934 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 1 [320/1000 (32%)]	Loss: 0.048593
2022-01-04 10:33:42,107 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 1 [480/1000 (48%)]	Loss: 0.035933
2022-01-04 10:33:42,165 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Trai

2022-01-04 10:35:45,205 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 3 [960/1000 (95%)]	Loss: 0.039484
2022-01-04 10:35:45,477 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 3 [960/1000 (95%)]	Loss: 0.034735
2022-01-04 10:35:45,873 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 3 [960/1000 (95%)]	Loss: 0.040313
2022-01-04 10:35:48,361 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - results uploaded successfully 
2022-01-04 10:35:48,683 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - results uploaded successfully 
2022-01-04 10:35:49,284 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - results uploaded successfully 
2022-01-04 10:35:58,698 fedbiomed INFO - Downloading model params after training on node_6db91b12-7507-4dd4-b3ea-788da19cc6ed - from http://localhost:8844

2022-01-04 10:35:59,604 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_2f6f9985-2136-4ced-b2b9-ba6cb4a5ecfc', 'job_id': 'f023bb56-34ec-4ba0-bee8-e6d043f45388', 'params_url': 'http://localhost:8844/media/uploads/2022/01/04/researcher_params_1207c487-25ba-43bc-b783-227b88170087.pt', 'training_args': {'batch_size': 16, 'lr': 1e-05, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'training_data': {'node_6db91b12-7507-4dd4-b3ea-788da19cc6ed': ['dataset_4d619920-dd9c-4f6a-a308-055ba0c6eed1']}, 'model_args': {}, 'model_url': 'http://localhost:8844/media/uploads/2022/01/04/my_model_78e85ccc-2b2c-40ee-9a8c-b6befbeef531.py', 'model_class': 'MyMonaiTrainingPlan', 'command': 'train'}
2022-01-04 10:35:59,920 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - {'monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x15a3e3fd0>, 'batch_size': 16, 'lr': 1e-05, 'epochs': 3, 'dry

2022-01-04 10:37:21,274 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 3 [0/1000 (0%)]	Loss: 0.034923
2022-01-04 10:37:21,376 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 3 [0/1000 (0%)]	Loss: 0.037482
2022-01-04 10:37:22,136 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 3 [0/1000 (0%)]	Loss: 0.036704
2022-01-04 10:37:27,329 fedbiomed INFO - log from: node_d1557b63-00bd-49f8-83e2-ba59f8ef89e6 / INFO - Train Epoch: 3 [160/1000 (16%)]	Loss: 0.033940
2022-01-04 10:37:27,417 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 3 [160/1000 (16%)]	Loss: 0.037903
2022-01-04 10:37:28,216 fedbiomed INFO - log from: node_6db91b12-7507-4dd4-b3ea-788da19cc6ed / INFO - Train Epoch: 3 [160/1000 (16%)]	Loss: 0.037524
2022-01-04 10:37:33,366 fedbiomed INFO - log from: node_15981cb3-39ef-4c54-9e72-2432d023491f / INFO - Train Epoch: 

## Testing


Once the federated model is obtained, it is possible to test it locally on an independent testing partition.
The test dataset is available at this link:

https://drive.google.com/file/d/1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD/

Following the Monai tutorial, in this section we will create a set of previously unseen pairs of moving vs fixed hands, and use the final federated model to predict the transformation between each pair.

In [ ]:
import os
import tempfile
import PIL
import torch
import numpy as np
import matplotlib.pyplot as plt
import gdown
import zipfile

print_config()
set_determinism(42)

Download the testing dataset on the local temporary folder.

In [ ]:
from copy import deepcopy
resource = "https://drive.google.com/uc?id=1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD"
base_dir = tmp_dir_model.name 

test_file = os.path.join(base_dir, "MedNIST_testing.zip")
data_dir = deepcopy(base_dir)
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
gdown.download(resource, test_file, quiet=False)

zf = zipfile.ZipFile(test_file)

for file in zf.infolist():
    zf.extract(file, data_dir)
data_dir = os.path.join(data_dir, "MedNIST_testing")
print(data_dir)

Create the testing data loader and pairs of moving vs fixed hands:

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

model = GlobalNet(
    image_size=(64, 64),
    spatial_dims=2,
    in_channels=2,  # moving and fixed
    num_channel_initial=16,
    depth=3).to(device)

if USE_COMPILED:
    warp_layer = Warp(3, "border").to(device)
else:
    warp_layer = Warp("bilinear", "border").to(device)

MedNISTDataset.dataset_folder_name = ""
test_data = MedNISTDataset(root_dir=data_dir, section="test", download=False, transform=None)
testing_datadict = [
    {"fixed_hand": item["image"], "moving_hand": item["image"]}
    for item in test_data.data if item["label"] == 4  # label 4 is for xray hands
]
test_transforms = Compose(
            [
                LoadImageD(keys=["fixed_hand", "moving_hand"]),
                EnsureChannelFirstD(keys=["fixed_hand", "moving_hand"]),
                ScaleIntensityRanged(keys=["fixed_hand", "moving_hand"],
                                     a_min=0., a_max=255., b_min=0.0, b_max=1.0, clip=True,),
                RandRotateD(keys=["moving_hand"], range_x=np.pi/4, prob=1.0, keep_size=True, mode="bicubic"),
                RandZoomD(keys=["moving_hand"], min_zoom=0.9, max_zoom=1.1, prob=1.0, mode="bicubic", align_corners=False),
                EnsureTypeD(keys=["fixed_hand", "moving_hand"]),
            ]
        )
val_ds = CacheDataset(data=testing_datadict[:1000], transform=test_transforms,
                      cache_rate=1.0, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=16, num_workers=0)

Create a model instance and assign to it the model parameters estimated at the last federated optimization round.
Generate predictions of the transformation between pairs.

In [ ]:
model = exp.model_instance
model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

for batch_data in val_loader:
    moving = batch_data["moving_hand"].to(device)
    fixed = batch_data["fixed_hand"].to(device)
    ddf = model(torch.cat((moving, fixed), dim=1))
    pred_image = warp_layer(moving, ddf)
    break

fixed_image = fixed.detach().cpu().numpy()[:, 0]
moving_image = moving.detach().cpu().numpy()[:, 0]
pred_image = pred_image.detach().cpu().numpy()[:, 0]

We can finally print some example of predictions from the testing dataset.

In [ ]:
%matplotlib inline
batch_size = 3
plt.subplots(batch_size, 3, figsize=(8, 10))
for b in range(batch_size):
    # moving image
    plt.subplot(batch_size, 3, b * 3 + 1)
    plt.axis('off')
    plt.title("moving image")
    plt.imshow(moving_image[b], cmap="gray")
    # fixed image
    plt.subplot(batch_size, 3, b * 3 + 2)
    plt.axis('off')
    plt.title("fixed image")
    plt.imshow(fixed_image[b], cmap="gray")
    # warped moving
    plt.subplot(batch_size, 3, b * 3 + 3)
    plt.axis('off')
    plt.title("predicted image")
    plt.imshow(pred_image[b], cmap="gray")
plt.axis('off')
plt.show()